In [1]:
import Embedding_Model
import pandas as pd
import Processing as p
import torch
import torch.nn.functional as F
from transformers import BertTokenizer


[nltk_data] Downloading package wordnet to C:\Users\Andre
[nltk_data]     Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Andre
[nltk_data]     Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andre Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Andre
[nltk_data]     Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Andre
[nltk_data]     Italiano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Andre Italiano\AppData\Roaming\nltk_dat

In [2]:
# load data
data = pd.read_excel('UpdatedTenders.xlsx')

In [3]:
# initialize model and preprocesser 
model = Embedding_Model.Pretrained_model()
preprocessor = Embedding_Model.Textprocessor()

In [4]:
data['Description'] = data['Description'].apply(p.remove_html_tags)

In [5]:
data_list = list(data['Description'])

In [11]:
def preprocess_for_bert(data):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    x = tokenizer(text = data,return_tensors = 'pt')
    return x

def preprocess_and_forward(data):
    x = preprocess_for_bert(data)
    
    with torch.no_grad():
        hidden_out = model(x['input_ids'],x['attention_mask'],x['token_type_ids'])
    cls_token = hidden_out.last_hidden_state
    cls_token = cls_token[0][0]
    cls_token = cls_token.unsqueeze(0)
    return cls_token

def cosine_sim(x1,x2):
    return F.cosine_similarity(x1,x2)


In [15]:
y1 = preprocess_and_forward(data_list[0])
y2 = preprocess_and_forward(data_list[20])
cosine_sim(y1,y2)

tensor([0.7468])

In [ ]:
scores = []
best = []
for items in data_list:
    x1 = data_list[5]
    x2 = items
    y1 = preprocess_and_forward(x1)
    y2 = preprocess_and_forward(x2)
    a = cosine_sim(y1,y2)
    
    if a > 0.85:
        best.append(items)
    scores.append(a)

In [ ]:
best